# 数据预处理

In [1]:
import os
import random
data_path=r'plant-seedlings-classification'
# data_path=r'/kaggle/input/plant-seedlings-classification'  #for kaggle notebook
train_path=os.path.join(data_path,'train')
test_path=os.path.join(data_path,'test')

In [2]:
train_path,test_path

('plant-seedlings-classification\\train',
 'plant-seedlings-classification\\test')

In [3]:
def get_data_list(path,train_list_path,eval_list_path):
    
#     print("path=%s"%path)
    img_classes=os.listdir(path)

    train_list=[]
    eval_list=[]

    label=0
    cnt=0
    for img_class in img_classes:
        label_dict[str(label)]=img_class
#         print("{}:{}".format(label,img_class))

        img_class_path=os.path.join(path,img_class)
        imgs=os.listdir(img_class_path)
        for img in imgs:
            img_path=os.path.join(img_class_path,img)
            cnt+=1
            if cnt%10==0:
                eval_list.append("%s\t%s\n"%(img_path,str(label)))
#                 print("%s\t%s"%(img_path,str(label)))
            else:
                train_list.append("%s\t%s\n"%(img_path,str(label)))
#                 print("%s\t%s\n"%(img_path,str(label)))
        label+=1
    
    random.shuffle(train_list)
    with open(train_list_path,'w') as f:
        for line in train_list:
            f.write(line)
    
    with open(eval_list_path,'w') as f:
        for line in eval_list:
            f.write(line)

In [4]:
label_dict={}
get_data_list(train_path,"train_list.txt","eval_list.txt")

In [5]:
label_dict

{'0': 'Black-grass',
 '1': 'Charlock',
 '2': 'Cleavers',
 '3': 'Common Chickweed',
 '4': 'Common wheat',
 '5': 'Fat Hen',
 '6': 'Loose Silky-bent',
 '7': 'Maize',
 '8': 'Scentless Mayweed',
 '9': 'Shepherds Purse',
 '10': 'Small-flowered Cranesbill',
 '11': 'Sugar beet'}

# 定义Dataset

In [6]:
from PIL import Image 
import torch
import numpy as np
class Reader(torch.utils.data.Dataset):
    def __init__(self,path):
        super().__init__()
        
        self.img_paths=[]
        self.labels=[]
        
        with open(path,'r',encoding='utf-8') as f:
            for line in f.readlines():
                self.img_paths.append(line.strip().split('\t')[0])
                self.labels.append(int(line.strip().split('\t')[1]))

    def __getitem__(self,index):

        img_path=self.img_paths[index]
        label=self.labels[index]
        
        img=Image.open(img_path)
        if img.mode !='RGB':
            img=img.convert('RGB')
        img=img.resize((224,224),Image.BILINEAR)
        img=np.array(img).astype('float32')
        img=img.transpose((2,0,1))
        img/=255.0
        
        label=np.array([label],dtype='int64')
        return img,label
    
    def pt(self,index):
        print("路径:{}\t 标签值:{}".format(self.img_paths[index],self.labels[index]))

    def __len__(self):
        return len(self.img_paths)

C:\Users\lazybug\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
batch_size=4

In [8]:
train_dataset=Reader("train_list.txt")
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)


eval_dataset=Reader("eval_list.txt")
eval_loader=torch.utils.data.DataLoader(eval_dataset,batch_size=batch_size,shuffle=False)

In [9]:
train_dataset.pt(100)
print("训练集样本数:%i"%train_dataset.__len__())
print("测试集样本数:%i"%eval_dataset.__len__())
eval_dataset.pt(0)

路径:plant-seedlings-classification\train\Cleavers\98ad03ef1.png	 标签值:2
训练集样本数:4275
测试集样本数:475
路径:plant-seedlings-classification\train\Black-grass\0b3e7a7a9.png	 标签值:0


In [10]:
# for data in train_loader:
#     x,y = data
#     print(x.shape)
#     print(y.shape)
#     print(y)
#     print(x)
#     break

# 配置模型

In [11]:
class ConvPool(torch.nn.Module):
    def __init__(self,num_channels,num_filters,filter_size,pool_size,pool_stride,groups,conv_stride=1,conv_padding=1):
        super(ConvPool,self).__init__()

        for i in range(groups):
            self.add_module(
                "conv_%d"%i,
                torch.nn.Conv2d(
                    in_channels=num_channels,
                    out_channels=num_filters,
                    kernel_size=filter_size,
                    stride=conv_stride,
                    padding=conv_padding
                )
            )
            num_channels=num_filters # 循环中下次的输入通道和上次的输出通道数相同
            self.add_module(
                'BN_%d'%i,
                torch.nn.BatchNorm2d(num_channels)
            )
            self.add_module(
                'relu%d'%i,
                torch.nn.ReLU()
            )
            
        
        self.add_module(
            'Maxpool',
            torch.nn.MaxPool2d(
                kernel_size=pool_size,
                stride=pool_stride
            )
        )
    def forward(self,inputs):
        x=inputs
        for prefix,sub_layer in self.named_children():
            x=sub_layer(x)
        return x

In [12]:

class VGGNet(torch.nn.Module):
    def __init__(self):
        super(VGGNet,self).__init__()
        self.convpool01=ConvPool(3,64,3,2,2,2)
        self.convpool02=ConvPool(64,128,3,2,2,2)
        self.convpool03=ConvPool(128,256,3,2,2,3)
        self.convpool04=ConvPool(256,512,3,2,2,3)
        self.convpool05=ConvPool(512,512,3,2,2,3)

        self.fc01=torch.nn.Linear(512*7*7,4096)
        self.fc02=torch.nn.Linear(4096,4096)
        self.fc03=torch.nn.Linear(4096,12)

    def forward(self,inputs):
        # print('input_size:{}'.format(inputs.shape))
        
        x=self.convpool01(inputs)
#         print(self.convpool01.state_dict())
        x=self.convpool02(x)
        x=self.convpool03(x)
        
        
        
        x=self.convpool04(x)
        x=self.convpool05(x)
        
        x=torch.reshape(x,[-1,512*7*7])
        x=self.fc01(x)
        x=self.fc02(x)
        y=self.fc03(x)
        return y

In [13]:
class test_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convpool01=ConvPool(3,64,3,2,2,2)
        self.convpool02=ConvPool(64,128,3,2,2,2)
        self.fc01=torch.nn.Linear(128*56*56,12)
        self.fc02=torch.nn.Linear(12,12)
        self.fc03=torch.nn.Linear(12,12)
        
    
    def forward(self,x):
        x=self.convpool01(x)
        x=self.convpool02(x)
        x=self.fc01(x.reshape([-1,128*56*56]))
        x=self.fc02(x)
        x=self.fc03(x)
        return x

In [14]:
model=VGGNet()
# model=test_model()
print(model)


VGGNet(
  (convpool01): ConvPool(
    (conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BN_0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU()
    (conv_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BN_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (Maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convpool02): ConvPool(
    (conv_0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BN_0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU()
    (conv_1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (BN_1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (Maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, c

In [15]:
def fit(epochs):
    device = torch.device('cuda')
    model.to(device)
    model.train()

    cross_entropy=torch.nn.CrossEntropyLoss()
    opt=torch.optim.Adam(params=model.parameters(),lr=0.0001)

    steps=0
    maxacc=0
    for epo in range(epochs):
        for batch_id,data in enumerate(train_loader):
            opt.zero_grad()
            x=data[0].to(device)
            y=data[1].to(device)
            pred=model(x)
            print(pred.shape,y.shape)
            loss=cross_entropy(pred,y.squeeze())
            res = torch.argmax(pred,1)
#             print(res,y.squeeze())
            acc = ((torch.eq(res,y.squeeze())).sum()).item()/res.shape[0]
#             acc=(pred==y).mean()
            loss.backward()
            opt.step()


            steps+=1
            if steps%10==0:
                print("epoch:{},step:{},loss:{},acc:{}".format(epo,steps,loss.item(),acc))
                losses=[]
                accs=[]
                for eval_data in eval_loader:
                    eval_x=eval_data[0].to(device)
                    eval_y=eval_data[1].to(device)
                    eval_pred=model(eval_x)
                    eval_loss=cross_entropy(eval_pred,eval_y.squeeze())
                    eval_res=torch.argmax(eval_pred,1)
                    eval_acc=((torch.eq(eval_res,eval_y.squeeze())).sum()).item()/eval_res.shape[0]
                    losses.append(float(eval_loss))
                    accs.append(float(eval_acc))
                    avg_acc=sum(accs)/len(accs)
                print("[Valid]  eval_loss:{},eval_acc:{}".format(sum(losses)/len(losses),avg_acc))
                    

                if maxacc<avg_acc:
                    maxacc=avg_acc
                    save_path='best.pd'
                    print('save to %s'%save_path)
                    torch.save(model.state_dict(),save_path)
            

        torch.save(model.state_dict(),"final.pd")

In [16]:
fit(epochs=2)

C:\Users\lazybug\AppData\Local\Temp\ipykernel_3516\119740133.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img=img.resize((224,224),Image.BILINEAR)


torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
epoch:0,step:10,loss:9.658984184265137,acc:0.25
[Valid]  eval_loss:16.85707255387131,eval_acc:0.1092436974789916
save to best.pd
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
torch.Size([4, 4096]) torch.Size([4, 1])
epoch:0,step:20,loss:5.698254108428955,acc:0.25


KeyboardInterrupt: 

# 保存模型

In [ ]:
torch.save(model.state_dict(),"final.pd")

# 加载模型

In [ ]:
state_path='final.pd'

class ConvPool(torch.nn.Module):
    def __init__(self,num_channels,num_filters,filter_size,pool_size,pool_stride,groups,conv_stride=1,conv_padding=1):
        super(ConvPool,self).__init__()

        for i in range(groups):
            self.add_module(
                "conv_%d"%i,
                torch.nn.Conv2d(
                    in_channels=num_channels,
                    out_channels=num_filters,
                    kernel_size=filter_size,
                    stride=conv_stride,
                    padding=conv_padding
                )
            )
            num_channels=num_filters # 循环中下次的输入通道和上次的输出通道数相同
            self.add_module(
                'BN_%d'%i,
                torch.nn.BatchNorm2d(num_channels)
            )
            self.add_module(
                'relu%d'%i,
                torch.nn.ReLU()
            )
            
        
        self.add_module(
            'Maxpool',
            torch.nn.MaxPool2d(
                kernel_size=pool_size,
                stride=pool_stride
            )
        )
    def forward(self,inputs):
        x=inputs
        for prefix,sub_layer in self.named_children():
            x=sub_layer(x)
        return x
    


class VGGNet(torch.nn.Module):
    def __init__(self):
        super(VGGNet,self).__init__()
        self.convpool01=ConvPool(3,64,3,2,2,2)
        self.convpool02=ConvPool(64,128,3,2,2,2)
        self.convpool03=ConvPool(128,256,3,2,2,3)
        self.convpool04=ConvPool(256,512,3,2,2,3)
        self.convpool05=ConvPool(512,512,3,2,2,3)

        self.fc01=torch.nn.Linear(512*7*7,4096)
        self.fc02=torch.nn.Linear(4096,4096)
        self.fc03=torch.nn.Linear(4096,12)

    def forward(self,inputs):
        # print('input_size:{}'.format(inputs.shape))
        
        x=self.convpool01(inputs)
#         print(self.convpool01.state_dict())
        x=self.convpool02(x)
        x=self.convpool03(x)
        
        
        
        x=self.convpool04(x)
        x=self.convpool05(x)
        
        x=torch.reshape(x,[-1,512*7*7])
        x=self.fc01(x)
        x=self.fc02(x)
        y=self.fc03(x)
        return x
    

    model=VGGNet()
    model.load_state_dict(torch.load(state_path))

# 预测

In [ ]:
def get_test_list(path,test_list_path):
    
    imgs=os.listdir(path)
    test_list=[]

    for img in imgs:
        img_path=os.path.join(path,img)
        test_list.append(img_path+"\n")
#         print(img_path)
    with open(test_list_path,'w') as f:
        for line in test_list:
            f.write(line)
            
            



from PIL import Image 
import torch
import numpy as np
class Test_Reader(torch.utils.data.Dataset):
    def __init__(self,path):
        super().__init__()
        
        self.img_paths=[]
        
        with open(path,'r',encoding='utf-8') as f:
            for line in f.readlines():
                self.img_paths.append(line.strip())
        self.img_paths.sort()

    def __getitem__(self,index):

        img_path=self.img_paths[index]
        
        img=Image.open(img_path)
        if img.mode !='RGB':
            img=img.convert('RGB')
        img=img.resize((224,224),Image.BILINEAR)
        img=np.array(img).astype('float32')
        img=img.transpose((2,0,1))
        img/=255.0
        
        return img,img_path


    def __len__(self):
        return len(self.img_paths)

get_test_list(test_path,"test_list.txt")
test_dataset=Test_Reader("test_list.txt")
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False)

device = torch.device('cuda')
with open('submit.csv','w') as f:
    f.write("file,species\n")
for test_data in test_loader:
    test_x=test_data[0].to(device)
    img_path=test_data[1][0]
    test_pred=model(test_x)
    test_res=torch.argmax(test_pred,1)
#     print(type(test_res.item()),type(img_path),test_res.item(),img_path.strip().split(r'/')[-1])
    with open('submit.csv','a') as f:
        f.write(img_path.strip().split(r'/')[-1]+","+label_dict[str(test_res.item())]+"\n")

In [ ]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())
print(os.listdir("/kaggle/working"))
from IPython.display import FileLink
FileLink('final.pt')